In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_csv('data/flipkart_com-ecommerce_sample.csv')
data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [3]:
data.sample()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
5244,5cb40c0a0cf232550c77a07717ba88d5,2015-12-01 12:40:44 +0000,http://www.flipkart.com/adimani-coneago-scarf-...,Adimani Coneago Scarf Fabric Necklace,"[""Jewellery >> Necklaces & Chains >> Necklaces""]",NKCE6T64KDBB4ZDG,3100.0,649.0,"[""http://img6a.flixcart.com/image/necklace-cha...",False,Adimani Coneago Scarf Fabric Necklace - Buy Ad...,No rating available,No rating available,Adimani,"{""product_specification""=>[{""key""=>""Collection..."


In [4]:
data.columns

Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')

In [5]:
data = data.drop(['pid', 'crawl_timestamp', 'product_url', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'product_rating','overall_rating','brand'],axis=1)

In [6]:
data.head()

,uniq_id,product_name,product_category_tree,description,product_specifications
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [7]:
data['uniq_id'].nunique()

20000

In [8]:
for i in range(0,20000):
    data['uniq_id'][i] = i+1

In [9]:
data.head()

,uniq_id,product_name,product_category_tree,description,product_specifications
0,1,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,"{""product_specification""=>[{""key""=>""Number of ..."
1,2,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,"{""product_specification""=>[{""key""=>""Installati..."
2,3,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,4,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,"{""product_specification""=>[{""key""=>""Number of ..."
4,5,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [10]:
data.dtypes

uniq_id                   object
product_name              object
product_category_tree     object
description               object
product_specifications    object
dtype: object

In [11]:
data['product_category_tree'][3]

'["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]'

In [12]:
def repair(text):
    text = str(text)
    pattern = re.compile('[\([{})\]]')
    text= pattern.sub(r'',text)
    text = text.replace('"','')
    text = text.replace(',','')
    text = text.replace('&','')
    pattern = re.compile('>>')
    return pattern.sub(r'',text)

In [13]:
repair('["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]')

"Clothing  Women's Clothing  Lingerie Sleep  Swimwear  Shorts  Alisha Shorts  Alisha Solid Women's Cycling Shorts"

In [14]:
data['product_category_tree'] = data['product_category_tree'].apply(repair)

In [15]:
data.head()

,uniq_id,product_name,product_category_tree,description,product_specifications
0,1,Alisha Solid Women's Cycling Shorts,Clothing Women's Clothing Lingerie Sleep Sw...,Key Features of Alisha Solid Women's Cycling S...,"{""product_specification""=>[{""key""=>""Number of ..."
1,2,FabHomeDecor Fabric Double Sofa Bed,Furniture Living Room Furniture Sofa Beds F...,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,"{""product_specification""=>[{""key""=>""Installati..."
2,3,AW Bellies,Footwear Women's Footwear Ballerinas AW Bel...,Key Features of AW Bellies Sandals Wedges Heel...,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,4,Alisha Solid Women's Cycling Shorts,Clothing Women's Clothing Lingerie Sleep Sw...,Key Features of Alisha Solid Women's Cycling S...,"{""product_specification""=>[{""key""=>""Number of ..."
4,5,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies Grooming Skin Coat Care Shamp...,Specifications of Sicons All Purpose Arnica Do...,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [16]:
data['description'][0]

"Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts"

In [17]:
data = data.drop('product_specifications',axis=1)

In [18]:
data.head()

,uniq_id,product_name,product_category_tree,description
0,1,Alisha Solid Women's Cycling Shorts,Clothing Women's Clothing Lingerie Sleep Sw...,Key Features of Alisha Solid Women's Cycling S...
1,2,FabHomeDecor Fabric Double Sofa Bed,Furniture Living Room Furniture Sofa Beds F...,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,3,AW Bellies,Footwear Women's Footwear Ballerinas AW Bel...,Key Features of AW Bellies Sandals Wedges Heel...
3,4,Alisha Solid Women's Cycling Shorts,Clothing Women's Clothing Lingerie Sleep Sw...,Key Features of Alisha Solid Women's Cycling S...
4,5,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies Grooming Skin Coat Care Shamp...,Specifications of Sicons All Purpose Arnica Do...


In [19]:
data['desc'] = data['product_category_tree']+data['description']

In [20]:
data['desc'][0]

"Clothing  Women's Clothing  Lingerie Sleep  Swimwear  Shorts  Alisha Shorts  Alisha Solid Women's Cycling ShortsKey Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts"

In [21]:
data = data.drop(['product_category_tree','description'],axis=1)

In [22]:
data.head()

,uniq_id,product_name,desc
0,1,Alisha Solid Women's Cycling Shorts,Clothing Women's Clothing Lingerie Sleep Sw...
1,2,FabHomeDecor Fabric Double Sofa Bed,Furniture Living Room Furniture Sofa Beds F...
2,3,AW Bellies,Footwear Women's Footwear Ballerinas AW Bel...
3,4,Alisha Solid Women's Cycling Shorts,Clothing Women's Clothing Lingerie Sleep Sw...
4,5,Sicons All Purpose Arnica Dog Shampoo,Pet Supplies Grooming Skin Coat Care Shamp...


In [23]:
data['desc']= data['desc'].str.lower()

In [24]:
def stemming(text):
    text = str(text)
    stemmer = PorterStemmer()
    return " ".join([stemmer.stem(word) for word in text.split()])

In [25]:
data.dtypes

uniq_id         object
product_name    object
desc            object
dtype: object

In [26]:
stemming("Clothing  Women's Clothing  Lingerie Sleep  Swimwear  Shorts  Alisha Shorts  Alisha Solid Women's Cycling ShortsKey Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts")

"cloth women' cloth lingeri sleep swimwear short alisha short alisha solid women' cycl shortskey featur of alisha solid women' cycl short cotton lycra navy, red, navy,specif of alisha solid women' cycl short short detail number of content in sale packag pack of 3 fabric cotton lycra type cycl short gener detail pattern solid ideal for women' fabric care gentl machin wash in lukewarm water, do not bleach addit detail style code altht_3p_21 in the box 3 short"

In [27]:
data['desc']= data['desc'].apply(stemming)

In [28]:
data['desc'][0]

"cloth women' cloth lingeri sleep swimwear short alisha short alisha solid women' cycl shortskey featur of alisha solid women' cycl short cotton lycra navy, red, navy,specif of alisha solid women' cycl short short detail number of content in sale packag pack of 3 fabric cotton lycra type cycl short gener detail pattern solid ideal for women' fabric care gentl machin wash in lukewarm water, do not bleach addit detail style code altht_3p_21 in the box 3 short"

In [29]:
data2 = data.iloc[:5000,:]

In [30]:
vect = TfidfVectorizer(max_features=5000,stop_words='english')

In [31]:
fitter = vect.fit_transform(data2['desc']).toarray()

In [32]:
vect.get_feature_names()

C:\Users\91700\Desktop\recommend\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '005',
 '01',
 '011',
 '012',
 '02',
 '020',
 '025',
 '03',
 '04',
 '047',
 '05',
 '050',
 '05tg',
 '069',
 '07',
 '08',
 '09',
 '099',
 '0d',
 '10',
 '100',
 '1000',
 '1007',
 '1008',
 '101',
 '1019',
 '102',
 '103',
 '1033',
 '1038',
 '104',
 '1042',
 '1049',
 '105',
 '105key',
 '106',
 '1070blk',
 '1071',
 '1079',
 '108',
 '1080p',
 '1099',
 '10ml',
 '11',
 '110',
 '1100',
 '111',
 '1110',
 '112',
 '1149',
 '1170',
 '1199',
 '12',
 '120',
 '1200',
 '122',
 '125',
 '1295',
 '1299',
 '12v',
 '13',
 '130',
 '131',
 '132',
 '1349',
 '135',
 '1350',
 '137',
 '1380',
 '139',
 '1390',
 '1399',
 '14',
 '140',
 '1400',
 '1445',
 '145',
 '1460',
 '147',
 '149',
 '1490',
 '1499',
 '14k',
 '15',
 '150',
 '1500',
 '1514',
 '152',
 '153',
 '1550',
 '1574',
 '159',
 '1590',
 '1599',
 '15cmx10cm',
 '16',
 '160',
 '164',
 '168',
 '169',
 '1695',
 '1699',
 '17',
 '170',
 '1700',
 '175',
 '1750',
 '179',
 '1790',
 '1795',
 '1799',
 '18',
 '180',
 '1800',
 '185',
 '189',
 '1899',
 '18k'

In [33]:
fitter[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
similarity = cosine_similarity(fitter)

In [36]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x: x[1])[1:6]

[(3, 0.9982197036822437),
 (6, 0.9956489017077094),
 (9, 0.9909875740274393),
 (13, 0.9885242348143827),
 (15, 0.9879090021202782)]

In [37]:
def recommend(product):
    product_index = data2[data2['product_name'] == product].index[0]
    distances = similarity[product_index]
    product_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x: x[1])[1:6]
    
    for x in product_list:
        print(data.iloc[x[0]].product_name)
        print(data.iloc[x[0]].desc)

In [38]:
s = 'FabHomeDecor Fabric Double Sofa Bed'
recommend(s)

FabHomeDecor Fabric Double Sofa Bed
furnitur live room furnitur sofa bed futon fabhomedecor fabric doubl sofa bed finish colo...fabhomedecor fabric doubl sofa bed (finish color - brown mechan type - pull out) price: rs. 22,646 • fine deep seat experi • save space with the all new click clack sofa bed • easi to fold and vice versa with simpl click clack mechan • chrome leg with mango wood frame for long term durabl • doubl cushion sofa bed to provid you with extra soft to make a fine seat experi • a doubl bed that can easili sleep two,specif of fabhomedecor fabric doubl sofa bed (finish color - brown mechan type - pull out) instal & demo instal & demo detail instal and demo for thi product is done free of cost as part of thi purchase. our servic partner will visit your locat within 72 busi hour from the deliveri of the product. in the box 1 sofa bed gener brand fabhomedecor mattress includ no deliveri condit knock down storag includ no mechan type pull out type sofa bed style contempora